In [4]:
import os
import sys
sys.path.insert(1, '../../../..')

In [5]:
import optuna
from optuna.trial import TrialState
from optuna.samplers import TPESampler

import torch
import torch.optim as optim
import torch.nn as nn

import numpy as np
import sklearn
import copy

from methods.DeepONet import DeepONet
from methods.methodsDataset.DeepONetDataset import DeepONetDataset
from methods.DataDrivenMethods import DDMethod
from solvers.Solver import Solver

In [6]:
params_solver = {'equation': 'Poisson', 'domain': [0., 1.], 'D': 1e-3, 'nx': 101}
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
params_DONUT ={
             'branch': {'layer_dims': None, 'activations': None, 'device': 'cpu', 'seed': 123}, 
             'trunk':{'layer_dims': None, 'activations': None, 'device': 'cpu', 'seed': 123}
             }

In [7]:
def delta(y, x, dy = 1., dx = 0.) :
    """    
    y : int, float or ndarray of size 1
    x : ndarray
    
    return dy if x = y and dx otherwise
    """
    if torch.is_tensor(y) or torch.is_tensor(y):
        return torch.where(x == y, dy, dx)
    return np.where(x == y, dy, dx)

In [8]:
params_solver['source_term'] = lambda y, x : delta(y, x)
params_solver['y'] = None

In [10]:
solver = Solver(params={'solver': params_solver})

In [11]:
Y_list = solver.x[20:81].reshape(-1, 1)

In [12]:
U_sols = []
for y in Y_list:
    solver.change_y(new_y= y)
    U_sols.append(solver.solve())
U_sols = np.stack(U_sols)

In [13]:
Y_train, Y_val, U_train, U_val = sklearn.model_selection.train_test_split(Y_list, U_sols, test_size=0.2, random_state=123)

In [14]:
delta_Y_train = delta(Y_train, solver.x)
delta_Y_val = delta(Y_val, solver.x)

In [16]:
def get_dataset():
    return delta_Y_train, delta_Y_val, U_train, U_val

In [17]:
def loss_fn(x, y = 0):
    return torch.square(y - x).mean()

In [36]:
def define_model(trial, input_size):
    nb_basis = trial.suggest_int("nb_basis", 8, 128)
    n_layers_branch = trial.suggest_int("n_layers_branch", 1, 5)
    branch_layers = [input_size]
    
    for i in range(n_layers_branch):
        out_features = trial.suggest_int("branch_units_l{}".format(i), 40, 120)
        branch_layers += [out_features]
    branch_layers += [nb_basis]

    n_layers_trunk = trial.suggest_int("n_layers_trunk", 1, 5)
    trunk_layers = [1]
    
    for i in range(n_layers_trunk):
        out_features = trial.suggest_int("trunk_units_l{}".format(i), 8, 40)
        trunk_layers += [out_features]
    trunk_layers += [nb_basis]
    
    params_DONUT_trial = copy.deepcopy(params_DONUT)
    params_DONUT_trial['branch']['layer_dims'] = branch_layers
    params_DONUT_trial['branch']['activations'] = 'tanh'
    params_DONUT_trial['trunk']['layer_dims'] = trunk_layers
    params_DONUT_trial['trunk']['activations'] = 'tanh'
 
    return DeepONet(params={'solver':params_solver, 'method':params_DONUT_trial})

In [37]:
def objective(trial):
    # Generate the model.
    model = define_model(trial, 101)

    # Generate the optimizers.
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)
 
    delta_Y_train, delta_Y_val, U_train, U_val = get_dataset()
    
    torch.manual_seed(params_DONUT['branch']['seed'])
    n_epochs = 10000
    hyperparameters = {'lr': lr, 'epochs': n_epochs, 'optimizer': 'Adam'}
    
    model.fit(hyperparameters=hyperparameters, 
              U_train=U_train, U_val=U_val, D_train=delta_Y_train, D_val=delta_Y_val)
        
    val_loss = model.loss_dict['val']
    trial.report(min(val_loss), n_epochs)
    return min(val_loss)

In [38]:
sampler = TPESampler(seed=params_DONUT['branch']['seed'])
study = optuna.create_study(direction="minimize", sampler=sampler)

[I 2023-08-10 18:18:31,090] A new study created in memory with name: no-name-a3f78049-b434-448e-b8b2-3fb4044b118f


In [ ]:
study.optimize(objective, n_trials=3)

[tr : 1.6e-03, val : 7.6e-01]: 100%|█████| 10000/10000 [00:16<00:00, 590.46it/s]
[I 2023-08-10 18:18:48,536] Trial 0 finished with value: 0.6443248987197876 and parameters: {'nb_basis': 92, 'n_layers_branch': 2, 'branch_units_l0': 58, 'branch_units_l1': 84, 'n_layers_trunk': 4, 'trunk_units_l0': 21, 'trunk_units_l1': 40, 'trunk_units_l2': 30, 'trunk_units_l3': 23, 'lr': 0.00015009027543233888}. Best is trial 0 with value: 0.6443248987197876.
[tr : 4.5e-02, val : 2.3e-01]: 100%|█████| 10000/10000 [00:15<00:00, 629.26it/s]
[I 2023-08-10 18:19:04,432] Trial 1 finished with value: 0.22322702407836914 and parameters: {'nb_basis': 49, 'n_layers_branch': 4, 'branch_units_l0': 75, 'branch_units_l1': 44, 'branch_units_l2': 72, 'branch_units_l3': 99, 'n_layers_trunk': 1, 'trunk_units_l0': 13, 'lr': 0.00039323677119884875}. Best is trial 1 with value: 0.22322702407836914.
[tr : 5.3e-02, val : 5.6e-01]:  19%|█     | 1861/10000 [00:03<00:13, 601.72it/s]

In [30]:
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  9.28167901292909e-06
  Params: 
    nb_basis: 19
    n_layers_branch: 4
    branch_units_l0: 22
    branch_units_l1: 9
    branch_units_l2: 21
    branch_units_l3: 32
    n_layers_trunk: 1
    trunk_units_l0: 13
    lr: 0.00039323677119884875


In [37]:
branch_activations = []
branch_layer_dims = []

trunk_activations = []
trunk_layer_dims = []

for key, value in trial.params.items():
    if key.split('_')[0] == 'branch':
        if key.split('_')[1] == 'activation':
            branch_activations.append(value)
        elif key.split('_')[1] == 'units':
            branch_layer_dims.append(value)
        
    elif key.split('_')[0] == 'trunk':
        if key.split('_')[1] == 'activation':
            trunk_activations.append(value)
        elif key.split('_')[1] == 'units':
            trunk_layer_dims.append(value)
    elif key == 'optimizer':
        optimizer = value
    elif key == 'lr':
        lr = value
    elif key == 'batch_size':
        batch_size = value
    elif key == 'nb_basis':
        basis_number = value

In [38]:
print('branch:\n', branch_activations, '\n', branch_layer_dims, 
      '\ntrunk:\n', trunk_activations, '\n', trunk_layer_dims,
      '\nhyperparameters\n',
      '\n', lr, '\n',  '\n', basis_number)

branch:
 [] 
 [22, 9, 21, 32] 
trunk:
 [] 
 [13] 
hyperparameters
 
 0.00039323677119884875 
 
 19
